<a href="https://colab.research.google.com/github/PatrickFRR/SantanderDataScience2023/blob/main/Desafio1_ReimaginandoETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander DataScience 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDS2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [63]:
from google.colab import files

uploaded = files.upload()

Saving SDS2023.csv to SDS2023 (1).csv


## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [64]:
import pandas as pd

df = pd.read_csv('SDS2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[21, 22, 23, 24, 27]


In [65]:
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

In [77]:
import requests
import json

# Função para obter informações do usuário por ID
def get_user(id):
    response = requests.get(f'{sdw2023_api_url}/users/{id}')
    # Verifica se a resposta foi bem-sucedida (código 200) e retorna os dados do usuário em formato JSON
    return response.json() if response.status_code == 200 else None

# Lista de IDs de usuário
user_ids = [21, 22, 23, 24, 27]

# Obtém informações dos usuários usando a função get_user
# Usa a sintaxe do operador walrus (:=) para atribuir o resultado da função e verificar se não é None
users = [user for id in user_ids if (user := get_user(id)) is not None]

# Imprime as informações dos usuários formatadas em JSON com indentação de 2 espaços
print(json.dumps(users, indent=2))


[
  {
    "id": 21,
    "name": "Pyterson",
    "account": {
      "id": 28,
      "number": "00012-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 500.0
    },
    "card": {
      "id": 21,
      "number": "**** **** **** 3232",
      "limit": 1000.0
    },
    "features": [],
    "news": [
      {
        "id": 266,
        "icon": "https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg",
        "description": "Eduardo esteve aqui"
      },
      {
        "id": 267,
        "icon": "https://images.genius.com/31fac7e2bef0ff83f25dbd8077863794.1000x1000x1.jpg",
        "description": "Eduardo esteve aqui"
      },
      {
        "id": 979,
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": "Pyterson, o que temos que ter sempre em mente \u00e9 que a cont\u00ednua expans\u00e3o de nossa atividade aponta para a melhoria das posturas dos \u00f3rg\u00e3os dirigentes com rela\u00

## **T**ransform

Gerando uma mensagem de marketing personalizada para cada usuário.

In [83]:
import random

# Função para gerar notícia aleatória com base em um usuário
def generate_news_from_message(user, message):
    # Substituir "{name}" no conteúdo da mensagem pelo nome do usuário
    message_with_name = message.replace("{name}", user['name'])
    return message_with_name

# Carregar as frases do arquivo
with open('mensagens.txt', 'r') as arquivo:
    frases = arquivo.read().splitlines()

# Função para gerar news usando a mensagem e adicionar à lista de news do usuário
def generate_and_add_news(user, message):
    news = generate_news_from_message(user, message)
    user['news'].append({
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
        "description": news
    })

# Gerar news para cada usuário
for user in users:
    user_id = user['id']
    message = random.choice(frases)  # Selecionar uma mensagem aleatória
    generate_and_add_news(user, message)
    print(f"Olá {user['name']} {message}")

# Exibir news geradas para cada usuário
for user in users:
    print(f"\nMensagens já enviadas para {user['name']}:\n")
    for news_item in user['news']:
        print(news_item['description'])
    print("\n")

Olá Pyterson simplifique suas transações internacionais com nossos serviços de câmbio e remessas globais.
Olá Caroline realize seus sonhos com nossos empréstimos flexíveis e taxas competitivas.
Olá Mar acumule pontos a cada transação e aproveite benefícios exclusivos com nosso programa de recompensas.
Olá Predu planeje sua aposentadoria com nossas soluções financeiras personalizadas. Seu futuro merece cuidado.

Mensagens já enviadas para Pyterson:

Eduardo esteve aqui
Eduardo esteve aqui
Pyterson, o que temos que ter sempre em mente é que a contínua expansão de nossa atividade aponta para a melhoria das posturas dos órgãos dirigentes com relação às suas atribuições. 
"Nossa equipe de especialistas está aqui para ajudar a realizar seus objetivos financeiros. Agende uma consulta hoje mesmo!"
"Acumule pontos a cada transação e aproveite benefícios exclusivos com nosso programa de recompensas."
"Faça seu dinheiro trabalhar por você. Descubra nossas opções de investimento hoje mesmo!"
"Simp

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [76]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Pyterson updated? True!
User Caroline updated? True!
User Mar updated? True!
User Predu updated? True!
